In [19]:
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch import nn
from torch.autograd import Function
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import cvxpy as cp
#from pyepo.model.opt import optModel


sys.path.insert(0, 'E:\\User\\Stevens\\Code\\The Paper\\algorithm')
from myutil import *
from features import get_all_features

# Suppress warnings
warnings.filterwarnings("ignore")

alpha, Q = 2, 20
df = pd.read_csv('data/data.csv')
# fix random seed for reproducibility

# report statistics on this dataset
df = df.sample(n=200, random_state=1)

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 'gagne_sum_tm1', 'gagne_sum_t', 
    'risk_score_percentile', 'screening_eligible', 'avoidable_cost_mapped', 'propensity_score', 'g_binary', 
    'g_continuous', 'utility_binary', 'utility_continuous'
]
# for race 0 is white, 1 is black
df_stat = df[columns_to_keep]
df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

# Replace all values less than 0.1 with 0.1
#df['risk_score_t'] = df['risk_score_t'].apply(lambda x: 0.1 if x < 0.1 else x)
df['g_continuous'] = df['g_continuous'].apply(lambda x: 0.1 if x < 0.1 else x)


risk = df['risk_score_t'].values
risk = risk + 0.001 if 0 in risk else risk


feats = df[get_all_features(df)].values
gainF = df['g_continuous'].values
decision = df['propensity_score'].values
cost = np.random.normal(1, 0.5, len(risk)).clip(0.1, 2)
race = df['race'].values

# transform the features
scaler = StandardScaler()
feats = scaler.fit_transform(feats)

from sklearn.model_selection import train_test_split
def AlphaFairness(util, alpha):
    if alpha == 1:
        return np.sum(np.log(util))
    elif alpha == 0:
        return np.sum(util)
    elif alpha == 'inf':
        return np.min(util)
    else:
        return np.sum(util**(1-alpha) / (1-alpha))

In [20]:
def solve_closed_form(g, r, c, alpha=alpha, Q=Q):

    g = g.detach().cpu().numpy() if isinstance(g, torch.Tensor) else g
    r = r.detach().cpu().numpy() if isinstance(r, torch.Tensor) else r
    c = c.detach().cpu().numpy() if isinstance(c, torch.Tensor) else c
    if c.shape != r.shape or c.shape != g.shape:
        raise ValueError("c, r, and g must have the same shape.")
    if np.any(c <= 0):
        raise ValueError("All cost values must be positive.")
    if np.any(r <= 0):
        raise ValueError("All risk values must be positive.")
    if np.any(g <= 0):
        raise ValueError("All gain factors must be positive.")
    
    n = len(c)
    utility = r * g
    
    if alpha == 0:
        ratios = utility / c
        sorted_indices = np.argsort(-ratios)  # Descending order
        d_star_closed = np.zeros(n)
        d_star_closed[sorted_indices[0]] = Q / c[sorted_indices[0]]
        
    elif alpha == 1:
        d_star_closed = Q / (n * c)
    
    elif alpha == 'inf':
        d_star_closed = (Q * c) / (utility * np.sum(c * c / utility))
        
    else:
        if alpha <= 0:
            raise ValueError("Alpha must be positive for general case.")

      #  numerator = np.power(c, -1/alpha) * np.power(utility, 1/alpha - 1)
      #  denominator = np.sum(numerator)
        
        numerator = np.power(c, -1/alpha) * np.power(utility, 1/alpha - 1)
        denominator = np.sum(np.power(c, 1-1/alpha) * np.power(utility, 1/alpha - 1))
        
        if denominator == 0:
            raise ValueError("Denominator is zero in closed-form solution.")
            
        d_star_closed = (numerator / denominator) * Q
    obj = AlphaFairness(d_star_closed * utility, alpha)
    return d_star_closed, obj

# Training

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import numpy as np

class RiskPredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc_out = nn.Linear(16, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softplus(self.fc_out(x)) + 0.001  # Ensure risk values remain positive
        return x


In [22]:
# Problem parameters
n = len(gainF)
x = cp.Variable(n)
p = cp.Parameter(n, nonneg=True)  # Fixed shape issue
Q = 100 # Budget constraint

# Define the CVXPY optimization problem
obj = cp.Maximize((1 / (1 - alpha)) * cp.sum(cp.power(cp.multiply(p, x), 1 - alpha)))
constr = [cp.sum(cp.multiply(cost, x)) <= Q, x >= 0]
problem = cp.Problem(obj, constr)

# Create CVXPY layer
cvxpylayer = CvxpyLayer(problem, parameters=[p], variables=[x])

In [23]:
true_risk = risk  # Your provided true risk from data
opt_sol, _ = solve_closed_form(gainF, risk, cost, alpha=alpha, Q=Q)

In [24]:
# Convert features and solutions into PyTorch tensors
tensor_feats = torch.tensor(feats, dtype=torch.float32)
tensor_true_sol = torch.tensor(opt_sol, dtype=torch.float32)
tensor_risk = torch.tensor(risk, dtype=torch.float32)  # True risk for regret calculation

In [25]:
def regret(pred_model, opt_model, feats, true_risk, gainF, cost, alpha=alpha, Q=Q):
    pred_model.eval()
    with torch.no_grad():
        pred_risk = pred_model(feats).squeeze().cpu().numpy()

    pred_risk = np.clip(pred_risk, 0.001, None)  # Ensure positivity
    gainF_np = gainF.flatten()
    cost_np = cost.flatten()
    true_risk_np = true_risk.cpu().numpy()

    # Compute optimal and predicted solutions
    opt_sol, opt_val = opt_model(gainF_np, true_risk_np, cost_np, alpha, Q)
    pred_sol, _ = opt_model(gainF_np, pred_risk, cost_np, alpha, Q)
    
    pred_obj = AlphaFairness(gainF_np * true_risk_np * pred_sol, alpha)

    # Compute regret
    normalized_regret = (opt_val - pred_obj) / (abs(opt_val) + 1e-7)

    pred_model.train()
    
    print ("In regret calculation: ", opt_val, pred_obj)
    return normalized_regret


In [26]:
lambda_fairness = 0


def AlphaFairnessTorch(util, alpha):
    # convert to torch tensor if not already
    if not isinstance(util, torch.Tensor):
        util = torch.tensor(util, dtype=torch.float32)
    if alpha == 1:
        return torch.sum(torch.log(util))
    elif alpha == 0:
        return torch.sum(util)
    elif alpha == 'inf':
        return torch.min(util)
    else:
        return torch.sum(util**(1-alpha) / (1-alpha))

In [27]:
gainT = torch.from_numpy(gainF)
# Fairness regularization parameter

# Initialize model and optimizer
model = RiskPredictor(input_dim=feats.shape[1])
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 50

true_util = gainT * tensor_risk * tensor_true_sol
opt_val = 1/(1-alpha) * (torch.sum(true_util ** (1-alpha)))

# Compute initial regret before training
initial_regret = regret(model, solve_closed_form, tensor_feats, tensor_risk, gainF, cost, alpha, Q)
print(f"Initial Regret: {initial_regret:.4f}")


for epoch in range(epochs):
    optimizer.zero_grad()

    # Predict risk using the neural network
    pred_risk = model(tensor_feats).squeeze()  # Ensure correct shape
    pred_risk = torch.clamp(pred_risk, min=0.001)  # Ensure risk is positive

    # Solve CVXPY optimization problem
    pred_sol, = cvxpylayer((gainT * pred_risk).unsqueeze(0))  # Add batch dimension fix
    
    # Compute MSE loss between predicted and true solutions
  #  loss = F.mse_loss(pred_sol, tensor_true_sol)
    
    # Compute normalized regret loss
    pred_util = gainT * true_risk * pred_sol
    pred_val = 1/(1-alpha) * (torch.sum(pred_util ** (1-alpha)))                       
    loss = (opt_val - pred_val) / (abs(opt_val) + 1e-7) 

    group0_mask = (race == 0)
    group1_mask = (race == 1)
    mse0 = torch.mean((pred_risk[group0_mask] - tensor_risk[group0_mask]) ** 2)
    mse1 = torch.mean((pred_risk[group1_mask] - tensor_risk[group1_mask]) ** 2)
    fairness_reg = torch.abs(mse0 - mse1)
    loss += lambda_fairness * fairness_reg

    # Backpropagation
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        # Verified that loss and regret are consistent with small numerical differences
        print ("From loss calculation: ", opt_val, pred_val)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
        
        regret_val = regret(model, solve_closed_form, tensor_feats, tensor_risk, gainF, cost, alpha, Q)
        print(f"Regret: {regret_val:.4f}")


# Compute final regret after training
final_regret = regret(model, solve_closed_form, tensor_feats, tensor_risk, gainF, cost, alpha, Q)
print(f"Final Regret: {final_regret:.4f}")

In regret calculation:  -246.87805537674768 -4187.582068880728
Initial Regret: 15.9621
From loss calculation:  tensor(-246.8781, dtype=torch.float64) tensor(-3742.1571, dtype=torch.float64, grad_fn=<MulBackward0>)
Epoch 10/50, Loss: 14.1579
In regret calculation:  -246.87805537674768 -3713.5770027433814
Regret: 14.0422
From loss calculation:  tensor(-246.8781, dtype=torch.float64) tensor(-3468.8391, dtype=torch.float64, grad_fn=<MulBackward0>)
Epoch 20/50, Loss: 13.0508
In regret calculation:  -246.87805537674768 -3450.570752511814
Regret: 12.9768
From loss calculation:  tensor(-246.8781, dtype=torch.float64) tensor(-3306.9461, dtype=torch.float64, grad_fn=<MulBackward0>)
Epoch 30/50, Loss: 12.3951
In regret calculation:  -246.87805537674768 -3292.159351899061
Regret: 12.3352
From loss calculation:  tensor(-246.8781, dtype=torch.float64) tensor(-3179.6928, dtype=torch.float64, grad_fn=<MulBackward0>)
Epoch 40/50, Loss: 11.8796
In regret calculation:  -246.87805537674768 -3168.425276386